In [2]:
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_community.agent_toolkits import FileManagementToolkit
from langchain_core.tools import Tool
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.models.openai._model_info import ModelInfo
from autogen_agentchat.agents import AssistantAgent
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage, MultiModalMessage

In [4]:
prompt ="""Your task is to find a one-way non-stop flight from JFK to LHR in June 2025.
First search online for promising deals.
Next, write all the deals to a file called flights.md with full details.
Finally, select the one you think is best and reply with a short summary.
Reply with the selected flight only, and only after you have written the details to the file."""
serper = GoogleSerperAPIWrapper(serper_api_key="b58a288fbb197c86cb34240b6058acb012c791fa")
langchain_serper = Tool(name="Internet_Search", func=serper.run, description="useful for when you need to search the internet")
autogen_serper=LangChainToolAdapter(langchain_serper)
autogen_tools=[autogen_serper]

langchain_file_management_tools=FileManagementToolkit(root_dir="sandbox").get_tools()
for tool in langchain_file_management_tools:
    autogen_tools.append(LangChainToolAdapter(tool))

for tool in autogen_tools:
    print(tool.name, tool.description)

Internet_Search useful for when you need to search the internet
copy_file Create a copy of a file in a specified location
file_delete Delete a file
file_search Recursively search for files in a subdirectory that match the regex pattern
move_file Move or rename a file from one location to another
read_file Read file from disk
write_file Write file to disk
list_directory List files and directories in a specified folder


In [19]:
import os
import markdown
from dotenv import load_dotenv
from IPython.display import display, Markdown
load_dotenv(override=True)
model_client = OpenAIChatCompletionClient(
        model="gemini-2.5-flash-lite", # Use gemini-1.5-flash if 2.0 is not yet in your tier
        api_key=os.getenv("GEMINI_API_KEY"),
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
        model_info=ModelInfo(
            family="gemini",
            max_input_tokens=200000,
            max_output_tokens=8192,
            mode="chat",
            vision=True,
            tool_call=True,
            function_calling=True,
            audio=False,
            json_output=True,
            structured_output=True,   # Added this line to fix UserWarning
            structured_outputs=True,  # Matches future required fields
            return_logprobs=False,
            parallel_tool_calls=True
)
)

In [ ]:
agent = AssistantAgent(name="searcher", model_client=model_client, tools=autogen_tools, reflect_on_tool_use=True)
message = TextMessage(content=prompt, source="user")
result = await agent.on_messages([message], cancellation_token=CancellationToken())
for message in result.inner_messages:
    
    print(message.content)
display(Markdown(result.chat_message.content))

[FunctionCall(id='function-call-13666607068730961721', arguments='{"query":"one-way non-stop flight JFK to LHR June 2025 deals"}', name='Internet_Search')]
[FunctionExecutionResult(content='Find flights to London Heathrow Airport from $174. Fly from New York John F Kennedy Airport on Aer Lingus, SWISS, Air Canada and more. Cheap Flights from New York (JFK) to London (LHR) start at $208 for one-way and $375 for round trip. Earn your airline miles on top of our rewards! Use Google Flights to find cheap flights from New York to London, starting at $330, and book tickets for your next trip. New York(JFK) to London(LHR) flight deals from $510* · Book our best prices between New York and London within the next 6 months · Flight tickets from New York to ... Find the best deals on flights from New York John F. Kennedy (JFK) to London Heathrow (LHR). Compare prices from hundreds of major travel agents and ... Search for cheap flights deals from JFK to LHR (John F. Kennedy Intl. to Heathrow). We

In [ ]:
message = TextMessage(content="OK proceed", source="user")
result = await agent.on_messages([message], cancellation_token= = CancellationToken())
for message in result.inner_messages:
    print(message.content)
display(Markdown(result.chat_message.content))
